In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
import seaborn as sns
import anndata
import scanpy as sc
import re
import decoupler as dc
import scvi
import warnings
import decoupler as dc

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")


/well/immune-rep/users/vbw431/conda/ivybridge/envs/scverse_new/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/well/immune-rep/users/vbw431/conda/ivybridge/envs/scverse_new/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /gpfs3/well/immune-rep/users/vbw431/conda/ivybridge/envs/scverse_new/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 0
/well/immune-rep/users/vbw431/conda/ivybridge/envs/scverse_new/lib/python3.8/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.reg

In [2]:
data_dir = "/well/immune-rep/users/vbw431/Projects/Peppa/data/"
newdata_dir = "/well/immune-rep/users/vbw431/Projects/Peppa/new_analysis/data/"
plot_dir = "/well/immune-rep/users/vbw431/Projects/Peppa/new_out/final_plots/"




In [3]:
##load data
adata = sc.read_h5ad("/well/immune-rep/users/vbw431/Projects/Peppa/out/peppa_azi_combat.h5ad")

labels = ["NK", "CD8", "CD4", "Bcells", "Myeloid_Plt"]
cluster_list = {}
scvi_list = {}
umap_list = {}
for i in [0,1,2,3,4]:
    print("reading in " + labels[i])
    cluster_list[labels[i]] = pd.read_csv(os.path.join(newdata_dir + labels[i] +"_clustering_final/", f"Peppa_{labels[i]}_cluster_assignment.csv"), index_col =0)
    scvi_list[labels[i]] = pd.read_csv(os.path.join(newdata_dir + labels[i] +"_embeddings/", f"Peppa_{labels[i]}_scvi.csv"), index_col =0)
    umap_list[labels[i]] = pd.read_csv(os.path.join(newdata_dir + labels[i] +"_embeddings/", f"Peppa_{labels[i]}_umap.csv"), index_col =0)

##generate adata_list
adata_list = {}

for i in [0,1,2,3, 4]:
    adata_list[labels[i]] = adata[cluster_list[labels[i]].index].copy()
    adata_list[labels[i]].obsm["X_scVI"] = scvi_list[labels[i]].loc[adata_list[labels[i]].obs_names].values
    adata_list[labels[i]].obsm["X_umap"] = umap_list[labels[i]].loc[adata_list[labels[i]].obs_names].values
    adata_list[labels[i]].obs = adata_list[labels[i]].obs.merge(cluster_list[labels[i]], left_index=True, right_index=True, how="inner")
    adata_list[labels[i]].layers['counts'] = adata_list[labels[i]].X.copy()
    sc.pp.normalize_total(adata_list[labels[i]], target_sum=1e4)
    sc.pp.log1p(adata_list[labels[i]])
    adata_list[labels[i]].layers['normalized'] = adata_list[labels[i]].X.copy()
    
##update meta_data with new clinical
clin_meta = pd.read_csv(newdata_dir + "index_demo.csv", index_col = 0)
clin_meta = clin_meta[["disease_group", 
                       "bio_replicate", 
                       "study_disease", 
                       "scanpy_index", 
                       "study_ID", 
                       "Treatment_status",
                      "Ethnicity",
                      "Sex",
                      "Age",
                      "HBV_serostatus",
                      "HBV_sAg_titre",
                      "HBV_DNA_VL"]]

new_obs = {}

for name in labels:
    del adata_list[name].obs["disease_group"]
    adata_list[name].obs["study_disease_orig"] = adata_list[name].obs["study_disease"].copy()
    del adata_list[name].obs["study_disease"]
    new_df = pd.merge(adata_list[name].obs, clin_meta, how='left', left_on = ['bio_replicate','scanpy_index'], right_on = ['bio_replicate','scanpy_index'])
    new_df.index = adata_list[name].obs.index
    new_obs[name] = new_df.copy()
    adata_list[name].obs = new_df.copy()

                 
for name in labels:
    adata_list[name].obs["celltype_consensus"] = str(name)

for name in labels:
    adata_list[name].obs["celltype_consensus.l1"] = str(name)
    adata_list[name].obs["celltype_consensus.l2"] = adata_list[name].obs[str(name+".annotation.l1")]
    adata_list[name].obs["celltype_consensus.l3"] = adata_list[name].obs[str(name+".annotation.l2")]
    
##concat
adata_all = anndata.concat(adata_list, join= "outer", index_unique=None)



reading in NK
reading in CD8
reading in CD4
reading in Bcells
reading in Myeloid_Plt


## Select treated groups and timepoint 0 from azimuth

In [4]:
##Assign CMV status based on azimuth preprint (https://doi.org/10.1016/j.cell.2021.04.048)

conditions = [
  ([re.search("_0|SeuratProject",a) is not None for a in adata_all.obs["orig.ident"]]),
  ([re.search("_0|SeuratProject",a) is None for a in adata_all.obs["orig.ident"]]),
 ]

# create a list of the values we want to assign for each condition
values = ["Subset", "Remove"]

# create a new column and use np.select to assign values to it using our lists as arguments
adata_all.obs["prevax"] = np.select(conditions, values)
adata_all = adata_all[adata_all.obs["prevax"] == "Subset"].copy()


In [5]:
NK = adata_all[adata_all.obs["celltype_consensus.l1"] == "NK"].copy()

In [6]:
np.max(NK.layers["counts"])
NK.X = NK.layers["counts"].copy()
sc.pp.normalize_total(NK, target_sum=1e4)
sc.pp.log1p(NK)

In [7]:
import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging
import rpy2.robjects.lib.ggplot2 as gp
from rpy2.robjects import pandas2ri
from rpy2.robjects import r
from rpy2.ipython.ggplot import image_png

#sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

In [8]:
%%R
.libPaths(c(paste0("/well/immune-rep/users/vbw431/conda/ivybridge/envs/scverse_new/r_modules"), .libPaths()))

library(tidyverse)
library(Seurat)
library(SCPA)
library(UCell)
cur.dir = "/well/immune-rep/users/vbw431/Projects/Peppa/data/DIMITRA_FASTQ/"
work.dir = "/well/immune-rep/users/vbw431/Projects/Peppa/"
out.dir = "/well/immune-rep/users/vbw431/Projects/Peppa/new_analysis/out/"
references = "/well/immune-rep/users/vbw431/reference/reference/refdata-gex-GRCh38-2020-A/"
com.out = "/well/immune-rep/users/vbw431/Projects/Peppa/reference_combat/"

##plotting
library(ggplot2); theme_set(theme_bw(base_size = 18)+
                              theme(strip.text = element_text(colour = 'black', face="bold",size=12), 
                                    panel.grid.major = element_blank(), 
                                    panel.grid.minor = element_blank(),
                                    panel.border = element_rect(size = 0.7),
                                    axis.ticks.length=unit(.10, "cm"),
                                    axis.ticks = element_line(size=0.7),
                                    strip.background = element_blank()))



-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.2     v readr     2.1.4
v forcats   1.0.0     v stringr   1.5.0
v lubridate 1.9.2     v tibble    3.2.1
v purrr     1.0.1     v tidyr     1.3.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x purrr::%@%()                 masks rlang::%@%()
x lazyeval::as_name()          masks rlang::as_name()
x lazyeval::call_modify()      masks rlang::call_modify()
x lazyeval::call_standardise() masks rlang::call_standardise()
x lazyeval::expr_label()       masks rlang::expr_label()
x lazyeval::expr_text()        masks rlang::expr_text()
x lazyeval::f_env()            masks rlang::f_env()
x lazyeval::f_env<-()          masks rlang::f_env<-()
x lazyeval::f_label()          masks rlang::f_label()
x lazyeval::f_lhs()            masks rlang::f_lhs()
x lazyeval::f_lhs<-()          masks rlang::f_lhs<-()
x lazyeval::f_rhs()            masks rlang::f_rhs()
x lazyeval::f_rhs<-() 

In [9]:
%%R 
#load data 

##load, normalise  and trim to remove pre_treatment replicates and post vaccine samples from azimuth

peppa <- readRDS(paste0(out.dir, "/peppa_pbmc_all.rds"))

##filter pre vaccination samples and pre treatment samples out
peppa <- subset(peppa, subset = Treatment_status != "Pre_treatment")

trim_barcodes <- peppa@meta.data %>% 
  dplyr::select(orig.ident) %>%
  dplyr::filter(grepl("_0|SeuratProject", orig.ident))
table(trim_barcodes)

peppa <- subset(peppa, cells = rownames(trim_barcodes))



In [10]:
%%R
Gran_list <- c("GZMB", "GZMA", "GZMK", "GZMM", "GZMH", "PRF1")



In [11]:
%%R
NK <- subset(peppa, subset = celltype_consensus.l1 == "NK")


In [12]:
%%R
##downsample evenly per cell subset per condition
set.seed(1)
adaptive_barcodes <- NK@meta.data %>%
mutate(cells = rownames(.)) %>%
filter(grepl("NK.Adaptive$", celltype_consensus.l2))%>%
group_by(celltype_consensus.l2, study_disease) %>%
sample_n(2000,replace=FALSE) %>% 
ungroup() %>% 
select(cells) %>% 
unlist()

set.seed(1)
adaptive_chemo_barcodes <- NK@meta.data %>%
mutate(cells = rownames(.)) %>%
filter(grepl("NK.Adaptive.CCL4.CCL3$", celltype_consensus.l2))%>%
group_by(celltype_consensus.l2, study_disease) %>%
sample_n(400,replace=FALSE) %>% 
ungroup() %>% 
select(cells) %>% 
unlist()

set.seed(1)
adaptive_mat_barcodes <- NK@meta.data %>%
mutate(cells = rownames(.)) %>%
filter(grepl("NK.Mature$", celltype_consensus.l2))%>%
group_by(celltype_consensus.l2, study_disease) %>%
sample_n(500,replace=FALSE) %>% 
ungroup() %>% 
select(cells) %>% 
unlist()

CD56_mat_barcodes <- NK@meta.data %>%
mutate(cells = rownames(.)) %>%
filter(grepl("NK.CD56bright$", celltype_consensus.l2))%>%
group_by(celltype_consensus.l2, study_disease) %>%
sample_n(300,replace=FALSE) %>% 
ungroup() %>% 
select(cells) %>% 
unlist()

Prolif_mat_barcodes <- NK@meta.data %>%
mutate(cells = rownames(.)) %>%
filter(grepl("NK.Prolifering$", celltype_consensus.l2))%>%
group_by(celltype_consensus.l2, study_disease) %>%
sample_n(90,replace=FALSE) %>% 
ungroup() %>% 
select(cells) %>% 
unlist()

downsample_cells <- c(adaptive_barcodes, adaptive_chemo_barcodes, adaptive_mat_barcodes, CD56_mat_barcodes, Prolif_mat_barcodes)

NK <- subset(NK, cells = downsample_cells)


In [13]:
%%R
NK_meta <- NK@meta.data


In [14]:
%%R
NK <- NormalizeData(NK)


In [18]:
%%R
subset <- NK@assays$RNA@data[rownames(NK@assays$RNA@data) %in% Gran_list, ]

Gene_meta <- cbind(NK_meta, t(as.matrix(subset)))

                               orig.ident nCount_RNA nFeature_RNA sample_ID
AAACCTGTCGCATGGC-1-HIVHBV_C SeuratProject   2508.859         1259  HIVHBV_C
AAACGGGAGATATACG-1-HIVHBV_C SeuratProject   2921.987         2053  HIVHBV_C
AAACGGGAGTTACCCA-1-HIVHBV_C SeuratProject   2819.482         2175  HIVHBV_C
AAACGGGCATCGGTTA-1-HIVHBV_C SeuratProject   2773.429         1649  HIVHBV_C
AAAGATGTCTCGCTTG-1-HIVHBV_C SeuratProject   2699.850         1657  HIVHBV_C
AAAGCAAAGCGTTCCG-1-HIVHBV_C SeuratProject   2513.127         1455  HIVHBV_C
                            nCount_HTO nFeature_HTO HTO_maxID HTO_secondID
AAACCTGTCGCATGGC-1-HIVHBV_C        260            2 Hashtag-2    Hashtag-1
AAACGGGAGATATACG-1-HIVHBV_C        733            2 Hashtag-2    Hashtag-1
AAACGGGAGTTACCCA-1-HIVHBV_C        570            2 Hashtag-2    Hashtag-1
AAACGGGCATCGGTTA-1-HIVHBV_C       1193            2 Hashtag-2    Hashtag-1
AAAGATGTCTCGCTTG-1-HIVHBV_C       3162            2 Hashtag-1    Hashtag-2
AAAGCAAAGCGTTCCG-1

AAACCTGTCGCATGGC-1-HIVHBV_C        <NA>        <NA>        NaN          NaN
AAACGGGAGATATACG-1-HIVHBV_C        <NA>        <NA>        NaN          NaN
AAACGGGAGTTACCCA-1-HIVHBV_C        <NA>        <NA>        NaN          NaN
AAACGGGCATCGGTTA-1-HIVHBV_C        <NA>        <NA>        NaN          NaN
AAAGATGTCTCGCTTG-1-HIVHBV_C        <NA>        <NA>        NaN          NaN
AAAGCAAAGCGTTCCG-1-HIVHBV_C        <NA>        <NA>        NaN          NaN
                            X_index n_genes_by_counts total_counts
AAACCTGTCGCATGGC-1-HIVHBV_C    <NA>               NaN          NaN
AAACGGGAGATATACG-1-HIVHBV_C    <NA>               NaN          NaN
AAACGGGAGTTACCCA-1-HIVHBV_C    <NA>               NaN          NaN
AAACGGGCATCGGTTA-1-HIVHBV_C    <NA>               NaN          NaN
AAAGATGTCTCGCTTG-1-HIVHBV_C    <NA>               NaN          NaN
AAAGCAAAGCGTTCCG-1-HIVHBV_C    <NA>               NaN          NaN
                            total_counts_mt pct_counts_mt
AAACCTGTCGCATGGC-

In [ ]:
%%R
library(RColorBrewer)
display.brewer.pal(n = 8, name = 'Set3')

brewer.pal(n = 8, name = "Set3")


In [ ]:
%%R
data_summary <- function(x) {
   m <- mean(x)
   ymin <- m-sd(x)
   ymax <- m+sd(x)
   return(c(y=m,ymin=ymin,ymax=ymax))
}

In [ ]:
%%R -w 500 -h 1200
library(rstatix)
library(ggbeeswarm)
##Cytotoxicity 
study_cols <-  c("NK.Adaptive"="#8DD3C7",
                 "NK.Prolifering"="#FFFFB3", 
                 "NK.Mature" = "#BEBADA",
                "NK.Adaptive.CCL4.CCL3" = "#FB8072",
                "NK.CD56bright"="#B3DE69")

NK_cyto_dunn <- Gene_meta %>%
filter(!grepl("Pre", Treatment_status)) %>%
filter(!grepl("Azimuth", Ref_lab)) %>%
select(celltype_consensus.l2, 
       GZMB, GZMA, GZMK, GZMM, GZMH, PRF1) %>%
reshape2::melt(id.vars=c("celltype_consensus.l2")) %>%
mutate(gene = variable) %>%
group_by(gene) %>% 
rstatix::dunn_test(value~celltype_consensus.l2) %>%
as.data.frame() %>%
add_xy_position(step.increase = 0.5)

NK_cyto_plot<-Gene_meta %>%
filter(!grepl("Pre", Treatment_status)) %>%
filter(!grepl("Azimuth", Ref_lab)) %>%
select(celltype_consensus.l2, 
       GZMB, GZMA, GZMK, GZMM, GZMH, PRF1) %>%
reshape2::melt(id.vars=c("celltype_consensus.l2")) %>% 
mutate(gene = variable) %>%
ggplot(aes(celltype_consensus.l2, value))+
geom_violin(aes(color=celltype_consensus.l2, fill=celltype_consensus.l2),outlier.alpha=0, alpha=0.5, scale="width")+
ggrastr::rasterise(geom_quasirandom(aes(fill=celltype_consensus.l2),shape=21,size=0.7, color="black",alpha=1, stroke =0.05))+
stat_summary(fun.data=data_summary)+
facet_grid(vars(gene))+
xlab("")+
ylab("Normalised Expression level")+
theme(axis.text.x = element_blank(), axis.ticks.x = element_blank())+
scale_fill_manual(values = study_cols)+ 
scale_color_manual(values = study_cols)+ 
ggpubr::stat_pvalue_manual(NK_cyto_dunn %>% filter(p.adj.signif!= "ns"))

print(NK_cyto_plot)



In [ ]:
%%R
pdf(file = "/well/immune-rep/users/vbw431/Projects/Peppa/new_out/final_plots/NK_granzyme.pdf",width = 7,height = 12)
plot(NK_cyto_plot)
dev.off()

In [ ]:
%%R
subset2 <- NK@assays$RNA@data[rownames(NK@assays$RNA@data) %in% c("CD2", "CD38", "SIGLEC7"), ]

Gene_meta2 <- cbind(NK_meta, t(subset2))
head(Gene_meta2)

In [ ]:
%%R -w 800 -h 1000

study_cols2 <-  c("CTRL"="#79AF97FF","HBV"="#DF8F44FF", "HBV_HIV" = "#B24745FF" )

NK_mark_dunn <- Gene_meta2 %>%
filter(!grepl("Pre", Treatment_status)) %>%
filter(!grepl("Azimuth", Ref_lab)) %>%
select(celltype_consensus.l2, study_disease,
       CD2, CD38, SIGLEC7) %>%
reshape2::melt(id.vars=c("celltype_consensus.l2","study_disease")) %>%
mutate(gene = variable) %>%
group_by(celltype_consensus.l2, gene) %>% 
rstatix::dunn_test(value~study_disease) %>%
as.data.frame() %>%
add_xy_position(step.increase = 0.12)

NK_mark_plot<-Gene_meta2 %>%
filter(!grepl("Pre", Treatment_status)) %>%
filter(!grepl("Azimuth", Ref_lab)) %>%
select(celltype_consensus.l2,study_disease, 
       CD2, CD38, SIGLEC7) %>%
reshape2::melt(id.vars=c("celltype_consensus.l2", "study_disease")) %>% 
mutate(gene = variable) %>%
ggplot(aes(study_disease, value))+
geom_violin(aes(color=study_disease, fill=study_disease),outlier.alpha=0, alpha=0.5, scale="width")+
ggrastr::rasterise(geom_quasirandom(aes(fill=study_disease),shape=21,size=0.7, color="black",alpha=1, stroke =0.05))+
stat_summary(fun.data=data_summary)+
facet_grid(vars(celltype_consensus.l2), vars(gene))+
xlab("")+
ylab("Normalised Expression level")+
theme(axis.text.x = element_blank(), axis.ticks.x = element_blank())+
scale_fill_manual(values = study_cols2)+ 
scale_color_manual(values = study_cols2)+ 
ggpubr::stat_pvalue_manual(NK_mark_dunn %>% filter(p.adj.signif!= "ns"))




In [ ]:
%%R -w 700 -h 1000

print(NK_mark_plot)

pdf(file = "/well/immune-rep/users/vbw431/Projects/Peppa/new_out/final_plots/NK_CD38_CD2_SIGLEC7.pdf",width = 8,height = 12)
plot(NK_mark_plot)
dev.off()

In [19]:
clin_meta

,disease_group,bio_replicate,study_disease,scanpy_index,study_ID,Treatment_status,Ethnicity,Sex,Age,HBV_serostatus,HBV_sAg_titre,HBV_DNA_VL
1.0,HBV-Rx,patient_12,HBV,PBMC_NA,AM64.3,On_treatment,White_caucasian,M,55.0,sAg+:eAg:eAb+,307.000000,0.0
2.0,HBV-Rx,patient_11,HBV,HBV_C_Hashtag.1,CB75.3,On_treatment,BAME,M,44.0,sAg+:eAg-:eAb+,279.000000,22.0
3.0,CTRL-Prep,control_1,CTRL,HBV_C_Hashtag.2,J2067,On_PrEP,White_caucasian,M,46.0,Not_available,NaN,NaN
4.0,HBV_HIV-Rx,patient_10,HBV_HIV,HIVHBV_C_Hashtag.1,RFH-HB004,On_treatment,BAME,M,44.0,sAg+:eAg-:eAb+,208.000000,0.0
5.0,CTRL-Prep,control_1,CTRL,HIVHBV_C_Hashtag.2,J2067,On_PrEP,White_caucasian,M,46.0,Not_available,NaN,NaN
6.0,HBV-Pre,patient_1,HBV,HBV_Tre_1_Hashtag.1,1391,Pre_treatment,White_caucasian,F,55.0,sAg+:eAg-:eAb+,2479.000000,3917.0
7.0,HBV-Rx,patient_1,HBV,HBV_Tre_1_Hashtag.2,1522,On_treatment,White_caucasian,F,56.0,sAg+:eAg-:eAb+,2544.000000,20.0
8.0,HBV-Pre,patient_2,HBV,HBV_Tre_2_Hashtag.1,1380,Pre_treatment,BAME,M,34.0,sAg+:eAg-:eAb+,7303.000000,1381.0
9.0,HBV-Rx,patient_2,HBV,HBV_Tre_2_Hashtag.2,1564,On_treatment,BAME,M,35.0,sAg+:eAg-:eAb+,7765.000000,20.0
10.0,HBV-Pre,patient_3,HBV,HBV_Tre_3_Hashtag.1,1639,Pre_treatment,Chinese,M,53.0,sAg+:eAg-:eAb+,27.000000,9033.0
